
Para variables numéricas:

ANOVA (F-Statistic y p-valor): Para verificar si hay diferencias significativas en la media de la variable numérica entre las 7 categorías.

Gini impurity: Utilizando un árbol de decisión para ver qué tan bien la variable segmenta la variable objetivo.

Kruskal-Wallis: Alternativa no paramétrica a ANOVA cuando la normalidad no está garantizada.

Para variables categóricas:

Chi-cuadrado (χ² y p-valor): Para medir independencia entre la variable categórica y la variable objetivo.

IV (Information Value): Para medir la capacidad de la variable para discriminar entre categorías.

Entropía de Shannon: Para evaluar la incertidumbre de la variable respecto a la variable objetivo.

Cramer’s V: Para medir la fuerza de asociación entre la variable y la variable objetivo.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway, chi2_contingency, entropy, kruskal
from sklearn.tree import DecisionTreeClassifier

def calculate_gini(x, y):
    """Calcula la importancia de Gini usando un árbol de decisión simple."""
    if x.nunique() > 1:  # Solo calcular si hay más de un valor único
        clf = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42)
        clf.fit(x.values.reshape(-1, 1), y)
        return clf.feature_importances_[0]
    return np.nan

def information_value(x, y):
    """Calcula el Information Value (IV) para una variable categórica."""
    df = pd.DataFrame({'x': x, 'y': y})
    grouped = df.groupby('x')['y'].value_counts(normalize=True).unstack().fillna(0)
    
    # Verificar si hay información suficiente
    if grouped.shape[1] < 2:  
        return np.nan
    
    woe = np.log((grouped + 0.0001) / (1 - grouped + 0.0001))  # WoE
    iv = (grouped - (1 - grouped)) * woe
    return iv.sum().sum()

def cramers_v(x, y):
    """Calcula el coeficiente de Cramer para medir la asociación entre variables categóricas."""
    contingency_table = pd.crosstab(x, y)
    
    if contingency_table.shape[0] < 2 or contingency_table.shape[1] < 2:
        return np.nan  # No se puede calcular si hay muy pocas categorías
    
    chi2_val, _, _, _ = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape) - 1
    return np.sqrt(chi2_val / (n * min_dim))

def analyze_features(df, target_column):
    results = []
    y = df[target_column]  # La variable objetivo se mantiene categórica

    for column in df.columns:
        if column == target_column:
            continue
        
        x = df[column]
        result = {'Variable': column, 'Tipo': str(x.dtype)}

        # Si más del 95% de los valores son NaN, descartamos la variable
        if x.isna().sum() / len(x) > 0.95:
            result.update({'ANOVA_F': np.nan, 'Chi2': np.nan, 'IV': np.nan, 
                           'Gini': np.nan, 'Cramer_V': np.nan, 'Entropía': np.nan})
            results.append(result)
            continue

        if np.issubdtype(x.dtype, np.number):  # Variables numéricas
            groups = [x[y == cat].dropna() for cat in y.unique()]
            unique_values_per_group = [len(set(g)) > 1 for g in groups]

            if len(groups) > 1 and any(unique_values_per_group):  # Evitar errores en ANOVA y Kruskal
                f_stat, p_value = f_oneway(*groups)
                kw_stat, kw_p_value = kruskal(*groups)
                result.update({'ANOVA_F': f_stat, 'ANOVA_p': p_value, 'Kruskal_H': kw_stat, 'Kruskal_p': kw_p_value})
            else:
                result.update({'ANOVA_F': np.nan, 'ANOVA_p': np.nan, 'Kruskal_H': np.nan, 'Kruskal_p': np.nan})
            
            result['Gini'] = calculate_gini(x.dropna(), y[x.notna()])

        else:  # Variables categóricas
            if x.nunique() < 50:  # Limitar para evitar problemas en tablas muy grandes
                contingency_table = pd.crosstab(x, y)
                
                if contingency_table.shape[0] > 1 and contingency_table.shape[1] > 1:  # Asegurar que hay suficientes datos
                    chi2_val, p, _, _ = chi2_contingency(contingency_table)
                    result.update({'Chi2': chi2_val, 'Chi2_p': p})
                    result['IV'] = information_value(x, y)
                    result['Cramer_V'] = cramers_v(x, y)
                    result['Entropía'] = entropy(contingency_table.sum(axis=1), base=2)
                else:
                    result.update({'Chi2': np.nan, 'Chi2_p': np.nan, 'IV': np.nan, 'Cramer_V': np.nan, 'Entropía': np.nan})
            else:
                result.update({'Chi2': np.nan, 'Chi2_p': np.nan, 'IV': np.nan, 'Cramer_V': np.nan, 'Entropía': np.nan})

        results.append(result)

    return pd.DataFrame(results)



In [ ]:
# Convertir categóricas a numéricas usando one-hot encoding
if tipo_variable == "categórica":
    X_encoded = pd.get_dummies(X, drop_first=True)  # Convertir en dummies
else:
    X_encoded = X  # Dejar las numéricas sin cambios

# Verificar que la variable no tenga solo un valor único (para evitar errores en el modelo)
if X_encoded.nunique().values[0] > 1:  
    model = DecisionTreeClassifier(max_depth=1, random_state=42)
    model.fit(X_encoded, y)

    gini = 2 * model.tree_.impurity[0]  # Cálculo del índice de Gini
    y_pred = model.predict(X_encoded)

    accuracy_por_clase = {}
    recall_por_clase = {}

    for clase in clases:
        mascara = (y == clase)
        if mascara.sum() > 0:  # Evitar divisiones por cero
            accuracy_por_clase[clase] = accuracy_score(y[mascara], y_pred[mascara])
            recall_por_clase[clase] = recall_score(y[mascara], y_pred[mascara], average='macro', zero_division=0)
